# Nuclio - Trigger Kubeflow Pipeline

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
import os
import datetime

In [3]:
# nuclio: ignore
spec = nuclio.ConfigSpec(config={"spec.triggers.inference.kind":"cron",
                                "spec.triggers.inference.attributes.interval" :"12h",
                                "spec.readinessTimeoutSeconds" : 60,
                                "spec.minReplicas" : 1,
                                },
                         env={'PYTHON_SCRIPT' : '/kv-to-parquet.py',
                              'SCRIPTS_LOCATION' : '/User/igzshare/nuclio_trigger_kfp',
                              'V3IO_HOME' : os.getenv('V3IO_HOME'),
                              'V3IO_ACCESS_KEY' : os.getenv('V3IO_ACCESS_KEY'),
                              'V3IO_USERNAME' : os.getenv('V3IO_USERNAME'),
                              'V3IO_HOME_URL' : os.getenv('V3IO_HOME_URL'),
                              
                         },
                         cmd=['pip install mlrun v3io'],
                        ).with_v3io().add_volume('/User',os.path.join('/users',os.getenv('V3IO_USERNAME')),kind='v3io',name='v3io')

### Kubeflow pipeline.

In [4]:
#%run kubeflow-pipeline.ipynb

# Deploy

In [10]:
# nuclio: ignore
#%nuclio deploy -p netops -n generator -c
addr = nuclio.deploy_file(source='kubeflow-pipeline.ipynb',name='ktptriggeronejob',project='kfptriggers',spec=spec)


[nuclio] 2020-04-24 15:33:55,047 (info) Build complete
[nuclio] 2020-04-24 15:34:03,191 (info) Function deploy complete
[nuclio] 2020-04-24 15:34:03,207 done updating ktptriggeronejob, function address: 3.12.248.124:30005


8cfd5204-84db-11ea-9e14-00155d82b336

## Create basic auth for function

Generate an http password file
htpasswd -c ./auth **kfpuser**

Kubernetes config

kubectl -n default-tenant create secret generic **kfpuser** --from-file auth